In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %%bash
# set -m
# git clone https://github.com/sachdevkartik/DeepLense.git
# cd DeepLense && git checkout kartik_contribution
# cd ..
# mv DeepLense/Transformers_Classification_DeepLense_Kartik_Sachdev/* .
# rm -rf DeepLense

In [3]:
# %%bash
# pwd

In [4]:
!nvidia-smi

Sun Jul 30 19:21:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   88C    P8    10W /  N/A |      5MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# %%bash
# pip3 install --upgrade -r requirements.txt

In [6]:
# !pip uninstall numpy

In [7]:
# !pip install numpy

In [8]:
from __future__ import print_function
import os
import time
import copy
import json
import yaml

import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
import torchvision
from typing import *

In [9]:
from utils.util import *
from config.data_config import DATASET
from utils.dataset import DefaultDatasetSetupSSL
from self_supervised.losses.contrastive_loss import (
    ContrastiveLossEuclidean,
    ContrastiveLossEmbedding,
    SimCLR_Loss,
    NegativeCosineSimilarity,
)
from self_supervised.losses.sym_neg_cos_sim_loss import SymNegCosineSimilarityLoss

from models.modules.head import BYOLProjectionHead, BYOLPredictionHead
from utils.scheduler import cosine_schedule
from torch.utils.data import DataLoader, random_split


In [10]:
args = {
    "dataset_name": "Model_II",
    "save": "data",
    "num_workers": 8,
    "train_config_path": "self_supervised/config/resnet_byol.yaml",
    "cuda": True,
    "log_dir": "logger"
}

In [11]:
batch_size = 64

dataset_name = args["dataset_name"]
dataset_dir = args["save"]
use_cuda = args["cuda"]
num_workers = args["num_workers"]
train_config_path = args["train_config_path"]
log_dir_base = args["log_dir"]

In [12]:
classes = DATASET[f"{dataset_name}"]["classes"]
num_classes = len(classes)

# Open the YAML file and load its contents
with open(train_config_path, "r") as file:
    train_config = yaml.safe_load(file)

In [13]:
epochs_pretrained = train_config["pretrained"]["num_epochs"]
epochs_finetuned = train_config["finetuned"]["num_epochs"]

learning_rate = train_config["optimizer_config"]["lr"]
margin = train_config["ssl"]["margin"]
num_channels = train_config["channels"]
temperature = train_config["ssl"]["temperature"]
network_type = train_config["network_type"]
image_size = train_config["image_size"]
optimizer_config = train_config["optimizer_config"]

backbone = train_config["backbone"]

make_directories([dataset_dir])
seed_everything(seed=42)

# logging
current_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
log_dir = f"{log_dir_base}/{current_time}"
init_logging_handler(log_dir_base, current_time)

# dump config in logger
with open(f"{log_dir}/config.json", "w") as fp:
    json.dump(train_config, fp)

# saving model path location
model_path_pretrained = os.path.join(
    f"{log_dir}/checkpoint",
    f"{network_type}_pretrained_{dataset_name}_{current_time}.pt",
)

model_path_finetune = os.path.join(
    f"{log_dir}/checkpoint",
    f"{network_type}_finetune_{dataset_name}_{current_time}.pt",
)

In [14]:
# setup default dataset
default_dataset_setup = DefaultDatasetSetupSSL(dir=None)
default_dataset_setup.setup(dataset_name=dataset_name)
default_dataset_setup.setup_transforms(image_size=image_size)

>>> Multiple Transforms
>>> Multiple Transforms


In [15]:
# trainset
train_dataset = default_dataset_setup.get_dataset(mode="train")
# default_dataset_setup.visualize_dataset(train_dataset)

Model_II dataset already exists
train data: 89104


In [16]:
# split in train and valid set
split_ratio = 0.25  # 0.25
valid_len = int(split_ratio * len(train_dataset))
train_len = len(train_dataset) - valid_len

train_dataset, val_set = random_split(train_dataset, [train_len, valid_len])

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
)

val_loader = DataLoader(
    dataset=val_set, batch_size=batch_size, shuffle=True, num_workers=num_workers
)

# Load test dataset
# testset = default_dataset_setup.get_dataset(mode="val")
# test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=True)

# size check
sample = next(iter(train_loader))
print("num of classes: ", num_classes)
print(sample[0].shape)

/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anacond

num of classes:  3
torch.Size([64, 1, 224, 224])


In [17]:
class BYOL2(nn.Module):
    def __init__(self, backbone, num_ftrs=512):
        super().__init__()

        self.backbone = backbone
        self.backbone[0] = nn.Conv2d(
            1, 64, kernel_size=7, stride=2, padding=3, bias=False
        )

        self.projection_head = BYOLProjectionHead(num_ftrs, 1024, 256)
        self.prediction_head = BYOLPredictionHead(256, 1024, 256)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        return p

    def forward_momentum(self, x):
        y = self.backbone_momentum(x).flatten(start_dim=1)
        z = self.projection_head_momentum(y)
        z = z.detach()
        return z


resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = BYOL2(backbone)

In [18]:
# Create pretrain model
resnet = torchvision.models.resnet34()
backbone = nn.Sequential(*list(resnet.children())[:-1])

num_ftrs_dict = {
    "resnet18": 512,
    "resnet34": 512,
    "resnet50": 2048,

}
model = BYOL2(backbone, num_ftrs=num_ftrs_dict["resnet34"])

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# summary(model, input_size=(1, 1, 224, 224), device="cuda")

########################## Pretraining #############################

# optimizer and loss function for pretrain
optimizer_pretrain = torch.optim.SGD(model.parameters(), lr=0.06)

# criterion
criterion = NegativeCosineSimilarity()

In [19]:
print("Starting Training")
for epoch in range(epochs_pretrained):
    total_loss = 0
    best_loss = float("inf")

    momentum_val = cosine_schedule(epoch, epochs_pretrained, 0.996, 1)
    for batch_idx, (x0, x1, label) in enumerate(train_loader):
        update_momentum(model.backbone, model.backbone_momentum, m=momentum_val)
        update_momentum(
            model.projection_head, model.projection_head_momentum, m=momentum_val
        )
        x0 = x0.to(device)
        x1 = x1.to(device)
        p0 = model(x0)
        z0 = model.forward_momentum(x0)
        p1 = model(x1)
        z1 = model.forward_momentum(x1)
        loss = 0.5 * (criterion(p0, z1) + criterion(p1, z0))
        total_loss += loss.detach()
        loss.backward()
        optimizer_pretrain.step()
        optimizer_pretrain.zero_grad()
        if batch_idx % 10 == 0:
            print(
                f"Epoch [{epoch}/{epochs_pretrained}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item()}"
            )

    if total_loss < best_loss:
        best_loss = total_loss
        best_model = copy.deepcopy(model)
        torch.save(best_model.state_dict(), model_path_pretrained)

    avg_loss = total_loss / len(train_loader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

Starting Training


/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anacond

Epoch [0/10], Batch [0/1045], Loss: -0.028086233884096146
Epoch [0/10], Batch [10/1045], Loss: -0.5477247834205627
Epoch [0/10], Batch [20/1045], Loss: -0.6095417737960815


In [ ]:
!nvidia-smi

# New Section